In [25]:
# "Проверка гипотезы: Разворот зависит от предыдущих курсов (вернее, разниц с текущей) и, возможно, volumes. 
# Задача  разбивается на следующие: 1) определение разворота 2) тестированию гипотезы 
# Пока я в процессе решеения первой задачи.    

In [26]:
from get_data import get_polo_data
import math
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn import metrics 
from sklearn import svm
from pandas import DataFrame
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import plotly.graph_objects as go
import numpy as np
from scipy.signal import argrelextrema


In [27]:
# РАБОТАЮ с пятиминутками! 
sd = get_polo_data(pair="USDT_BTC", frame=(8)*60*60*24,zoom='5MIN')
print("Download from polo",len(sd))
print("Errors values",len(sd)-len(sd.dropna()))

DEBUG:poloniex:Getting 2020-01-13 00:20:00 - 2020-01-13 00:24:34 USDT_BTC candles from Poloniex...
DEBUG:poloniex:Updating USDT_BTC database with 1 entrys...
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 999.83it/s]
DEBUG:poloniex:Getting USDT_BTC chart data from db


Download from polo 2304
Errors values 0


In [28]:
# Функция для расчета минимумов и максимумов 
def SetDataFrameMinMax(n_sosed, n_lags):   
    
    #В цикле инициализируются столбцы - исходные  параметры для маш обучения   
    for i in range(0, n_lags): #создаем столбцы, курc: разница между текущим и i (макс количество передается в функции)
        sd["Lag%s" % str(i+1)] = sd["open"]-sd["open"].shift(i+1)
        sd["Vol%s" % str(i+1)] = sd["volume"].shift(i+1)
    
    # здесь нахожу минимумы и максимумы. Еще потестю на твоей функции 
    sd['min'] = sd.iloc[argrelextrema(sd.open.values, np.less_equal, order=n_sosed)[0]]['open']
    sd['max'] = sd.iloc[argrelextrema(sd.open.values, np.greater_equal, order=n_sosed)[0]]['open']
    
    #дальше инициируем два столбца. extr - значение экстремума (для визульного контроля на графике) и is_extr=1 если экстремум
    #под экстремумом понимает разворот 
    for z, row in sd.iterrows():      
        sd.loc[z,'is_extr']=0
        if (np.isnan(sd.loc[z,'min'])==False):
            sd.loc[z,'extr']=sd.loc[z,'min']
            sd.loc[z,'is_extr']=1
        if (np.isnan(sd.loc[z,'max'])==False):
            sd.loc[z,'extr']=sd.loc[z,'max']
            sd.loc[z,'is_extr']=1
        
# Функция построения графика для проверки правильно ли определяются мин и макс           
def ShowMyFig2(): 
    #графика для контроля. Сейчас min и max не видно, так как совпадают с extr 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sd.index, y=sd['open'],mode='lines',name="open_val"))
    fig.add_trace(go.Scatter(x=sd.index, y= sd['min'],mode='markers',name="min"))
    fig.add_trace(go.Scatter(x=sd.index, y= sd['max'],mode='markers',name="max"))
    fig.add_trace(go.Scatter(x=sd.index, y= sd['extr'],mode='markers',name="extr"))
    fig.show()


In [29]:
n_sosed=20
n_lags=5  #сколько столбцов с Lag-ми создавать 

SetDataFrameMinMax(n_sosed,n_lags)
ShowMyFig2() 
print ("Найдено {} экстремумов из {} значений".format(sd['is_extr'].sum(),len(sd)))

Найдено 82.0 экстремумов из 2304 значений


In [30]:
# Общий для входных данных фрагмент 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


model_rfc = RandomForestClassifier(n_estimators = 70) #в параметре передаем кол-во деревьев
model_knc = KNeighborsClassifier(n_neighbors = 18) #в параметре передаем кол-во соседей
model_lr = LogisticRegression(penalty='l1', tol=0.01) 
model_svc = svm.SVC()

kfold=5
sd=sd.fillna(0)   #обнуляю is_nan чтобы работали функции маш обучения 

target=sd['is_extr']

In [31]:
# Тестим на разных моделях. Сначала прогоняем на всех параметрах (Volume и разница за 1,2,3 дня) 

mmodels = {  \
        'RandomForestClassifier'   : model_rfc,
        'KNeighborsClassifier'     : model_knc,
        'LogisticRegression'     : model_lr,
        'SVC'     : model_svc
     }

train=sd[['Lag1','Lag2','Lag3','Vol1','Vol2','Vol3']]

for key in mmodels:
    train_parm, test_parm, train_indic, test_indic = model_selection.train_test_split(
    train, target, test_size = 0.3, random_state = 0)
    y_pred = cross_val_predict(mmodels[key], train, target, cv = kfold)
    print ("{}: Определено {} экстремумов из {}".format(key,confusion_matrix(target, y_pred)[1,1],sd['is_extr'].sum())) 
#     print(confusion_matrix(target, y_pred),classification_report(target, y_pred))


RandomForestClassifier: Определено 0 экстремумов из 82.0
KNeighborsClassifier: Определено 0 экстремумов из 82.0
LogisticRegression: Определено 0 экстремумов из 82.0
SVC: Определено 0 экстремумов из 82.0


In [32]:
# Получилось плохо. Дальше потестил на разных вх данных, лучше всего на Lage за последний день    
mmodels = {  \
        'RandomForestClassifier'   : model_rfc,
        'KNeighborsClassifier'     : model_knc,
        'LogisticRegression'     : model_lr,
        'SVC'     : model_svc
     }

train=sd[['Lag1']]

for key in mmodels:
    train_parm, test_parm, train_indic, test_indic = model_selection.train_test_split(
    train, target, test_size = 0.3, random_state = 0)
    y_pred = cross_val_predict(mmodels[key], train, target, cv = kfold)
    print ("{}: Определено {} экстремумов из {}".format(key,confusion_matrix(target, y_pred)[1,1],sd['is_extr'].sum())) 
#     print(confusion_matrix(target, y_pred),classification_report(target, y_pred))


RandomForestClassifier: Определено 8 экстремумов из 82.0
KNeighborsClassifier: Определено 0 экстремумов из 82.0
LogisticRegression: Определено 0 экстремумов из 82.0
SVC: Определено 0 экстремумов из 82.0


In [33]:
#Здесь тестирую на параметрах маш обучения: Разница не большая 
for i in range(1,80):
    model_rfc = RandomForestClassifier(n_estimators = i) #в параметре передаем кол-во деревьев
    mmodels = {  \
        'RandomForestClassifier'   : model_rfc
     }

    # train=sd[['Lag1','Lag2','Lag3','Vol1','Vol2','Vol3']]
    train=sd[['Lag1']]

    for key in mmodels:
        train_parm, test_parm, train_indic, test_indic = model_selection.train_test_split(
        train, target, test_size = 0.3, random_state = 0)
        y_pred = cross_val_predict(mmodels[key], train, target, cv = kfold)
        print ("{}: {}: Определено {} экстремумов из {}".format(key,i,confusion_matrix(target, y_pred)[1,1],sd['is_extr'].sum())) 



RandomForestClassifier: 1: Определено 6 экстремумов из 82.0
RandomForestClassifier: 2: Определено 4 экстремумов из 82.0
RandomForestClassifier: 3: Определено 3 экстремумов из 82.0
RandomForestClassifier: 4: Определено 3 экстремумов из 82.0
RandomForestClassifier: 5: Определено 7 экстремумов из 82.0
RandomForestClassifier: 6: Определено 4 экстремумов из 82.0
RandomForestClassifier: 7: Определено 7 экстремумов из 82.0
RandomForestClassifier: 8: Определено 6 экстремумов из 82.0
RandomForestClassifier: 9: Определено 6 экстремумов из 82.0
RandomForestClassifier: 10: Определено 7 экстремумов из 82.0
RandomForestClassifier: 11: Определено 8 экстремумов из 82.0
RandomForestClassifier: 12: Определено 6 экстремумов из 82.0
RandomForestClassifier: 13: Определено 7 экстремумов из 82.0
RandomForestClassifier: 14: Определено 4 экстремумов из 82.0
RandomForestClassifier: 15: Определено 6 экстремумов из 82.0
RandomForestClassifier: 16: Определено 5 экстремумов из 82.0
RandomForestClassifier: 17: Опред

In [34]:
# Дальше две идеи. Первая - автоматом потестить все алгоритмы маш обучения на всех комбинациях входных  данных
# Вторая - поиграть с параметрами при определении экстремумов 
